In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

class ParticleSwarm():
    def __init__(self,  x_domain, v_max, cost_f, dimension, swarmsize, swarm=None,
                alpha = 0.72984, #0.72, 0.3925
                beta = 2.05, #1.5, 2.5586, 2.05
                gamma = 2.05): #1.5, 1.3358, 2.05
        self.dimension = dimension
        self.swarmsize = swarmsize
        self.cost_f = cost_f
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        if x_domain == np.inf:
            self.x_domain = 10
        else:    
            self.x_domain = x_domain
        self.v_max = v_max
        if swarm is None:
            # np.random.seed(63)
            self.swarm = np.random.uniform(-self.x_domain, self.x_domain, (self.dimension, self.swarmsize))
        else:
            self.swarm = swarm
        self.bestLocal  = self.swarm.copy() # лучшие положения частиц
        # np.random.seed(63)
        self.bestGlobal = np.random.uniform(-self.x_domain, self.x_domain, (self.dimension, 1))
        self.velocity = np.random.uniform(-self.x_domain, self.x_domain, (self.dimension, self.swarmsize))
        self.f_best = self.cost_f(self.bestLocal) 
        self.best = self.f_best.min()
       

    def rate_particles(self):
        self.f_best = self.cost_f(self.bestLocal) 
        b = self.f_best >= self.cost_f(self.swarm) 
        self.bestLocal.T[b] = self.swarm.T[b] #из каждой строки "bestLocal" и "swarm" формируем массив с индексами из "b"

        k = np.argmin(self.f_best)
        self.bestGlobal = self.bestLocal.T[k].reshape(-1,1)
        self.best = self.f_best.min()
     
        r1, r2 = np.random.rand(2)
        self.velocity = self.alpha * self.velocity + self.beta * r1 * (self.bestLocal - self.swarm) + self.gamma * r2 * (self.bestGlobal - self.swarm)
        self.velocity = np.minimum(self.velocity,  self.v_max)
        self.velocity = np.maximum(self.velocity, -self.v_max)

        self.swarm = self.swarm + self.velocity #перемещение частицы
        self.swarm = np.minimum(self.swarm,  self.x_domain)
        self.swarm = np.maximum(self.swarm, -self.x_domain)

In [ ]:
class Jaya():
    def __init__(self, domain, cost_f, dimension, populationsize, population=None): 
        self.dimension = dimension
        self.populationsize = populationsize
        if domain == np.inf:
            self.domain = 10
        else:    
            self.domain = domain
        if population is None:
            # np.random.seed(63)
            self.population = np.random.uniform(-self.domain, self.domain, (self.dimension, self.populationsize))
        else:
            self.population = population
        self.population_updt  = self.population.copy() 
        self.cost_f = cost_f
        self.best = np.random.uniform(-self.domain, self.domain, (self.dimension, 1))
        self.worst = np.random.uniform(-self.domain, self.domain, (self.dimension, 1))
      
    def best_worst(self):
        f_value = self.cost_f(self.population)

        k = np.argmin(f_value)
        self.best = self.population.T[k].reshape(-1,1)

        q = np.argmax(f_value)
        self.worst = self.population.T[q].reshape(-1,1)

    def update(self):
        self.best_worst()
        r1, r2 = np.random.rand(2)
        self.population_updt = self.population + r1 * (self.best - abs(self.population)) - r2 * (self.worst - abs(self.population))

        b = self.cost_f(self.population_updt) <= self.cost_f(self.population) 
        self.population.T[b] = self.population_updt.T[b] 

        self.population = np.minimum(self.population,  self.domain)
        self.population = np.maximum(self.population, -self.domain)  

In [ ]:
#functions for optimization
def g(x):
    # return sum(x**2)
    # return sum(abs(x)) #f(0,0)=0 [-10, 10]
    # return 10 * x.shape[0] + sum(x**2 - 10 * np.cos(2* np.pi * x)) # Rastrigin f(0,..., 0)=0, -5.12 <= x <= 5.12
    # return  sum(x**2 - 10 * np.cos(2* np.pi * x) + 10) # Rastrigin f(0,..., 0)=0, -5.12 <= x <= 5.12
    # return 	np.e + 20 - 20 * np.exp(-0.2 * np.sqrt(1 / x.shape[0] * sum(x**2))) - np.exp(1 / x.shape[0] * sum(np.cos(2 * np.pi * x))) # -32 <= x <= 32, f(0, 0) = 0
    # return 418.9829 * x.shape[0] - sum(x * np.sin(np.sqrt(np.abs(x))))
    # return (x-1)**2 + 100 * (y - x**2)**2 #f(1,1)=0 Розенброк
    # return - np.cos(x[0]) * np.cos(x[1]) * np.exp( - ((x[0] - np.pi)**2 + (x[1] - np.pi)**2)) # Izom -100 <= x, y <= 100  f(pi ,pi )=-1
    return  -(x[1] + 47) * np.sin(np.sqrt(np.abs(x[0] / 2 + (x[1] + 47)))) - x[0] * np.sin(np.sqrt(np.abs(x[0] - (x[1] + 47)))) # "подставка для яиц" Eggholder function, f(512, 404.2319) = -959.6407, -512 <= x, y <= 512
    # return sum(-x * np.sin(np.sqrt(np.abs(x)))) # [-500, 500] f_min = -12569.5


In [ ]:
min_f = -959.6407 # минимум функции
dom = 512         # область определения
num_iter = 200    # количество итераций
num_sim = 50      # число симуляций
dim = 30          # размерность функции 
size = 20         # размер роя (количество частиц)

In [ ]:
all_err_pso = []
best_value_pso = []
for k in range(num_sim):
    mean_list_pso = []
    pso_multi = ParticleSwarm(x_domain=dom, v_max=np.inf, dimension=dim, swarmsize=size, cost_f=g, alpha=0.9, beta=1.5, gamma=0.25)
    for i in range(num_iter):
        pso_multi.rate_particles()
        minimum = np.abs(g(pso_multi.bestGlobal) - min_f)
        mean_list_pso.append(minimum)
    all_err_pso.append(mean_list_pso)
    best_value_pso.append(g(pso_multi.bestGlobal))


In [ ]:
all_err_jy = []
best_value_jy = []
for k in range(num_sim):
    mean_list_jy = []
    jy_one = Jaya(domain=dom, cost_f=g, dimension=dim, populationsize=size)
    jy_one.best_worst()
    for i in range(num_iter):
        jy_one.update()
        minimum = np.abs((g(jy_one.best) - min_f))
        mean_list_jy.append(minimum)
    all_err_jy.append(mean_list_jy)
    best_value_jy.append(g(jy_one.best))

In [ ]:
all_err_pso_jy = []
best_value_pso_jy = []
for k in range(num_sim):
    pso = ParticleSwarm(x_domain=dom, v_max=np.inf, dimension=dim, swarmsize=size, cost_f=g, alpha=0.9, beta=1.5, gamma=0.25)
    mean_list_pso_jaya = [np.abs(g(pso.bestGlobal) - min_f)]
    for i in range(num_iter):
        pso.rate_particles()
        if np.abs(g(pso.bestGlobal) - min_f) >= mean_list_pso_jaya[-1]:
            jy = Jaya(domain=pso.x_domain, cost_f=pso.cost_f, population=pso.swarm, dimension=pso.dimension, populationsize=pso.swarmsize)
            for j in range(5):
                jy.update()
            pso.swarm = jy.population
        mean_list_pso_jaya.append(np.abs(g(pso.bestGlobal) - min_f))
    all_err_pso_jy.append(mean_list_pso_jaya)
    best_value_pso_jy.append(g(pso.bestGlobal))

In [ ]:
def mean_all_simul(all_err):
    # all_err = np.asarray(all_err)
    mean_list = []
    for el in all_err.T:
        mean_list.append(el.mean())
    return np.asarray(mean_list)

In [ ]:
def print_statistical_analysis(all_best_scores):
    print("mean:", np.mean(all_best_scores.T[-1]))
    print("median:", np.median(all_best_scores.T[-1]))
    print("std:", np.std(all_best_scores.T[-1]))
    print("min", np.min(all_best_scores.T[-1]))

In [ ]:
all_err_pso = np.asarray(all_err_pso)
all_err_jy = np.asarray(all_err_jy)
all_err_pso_jy = np.asarray(all_err_pso_jy)
all_err_pso_jy = all_err_pso_jy.reshape(num_sim, 201)
print("--pso--")
print_statistical_analysis(all_err_pso)
print("--jaya--")
print_statistical_analysis(all_err_jy)
print("--pso_jaya--")
print_statistical_analysis(all_err_pso_jy[1:201])

In [ ]:
mean_list_pso = mean_all_simul(all_err_pso[0].T)
mean_list_jy = mean_all_simul(all_err_jy[0].T)
mean_list_pso_jaya = mean_all_simul(all_err_pso_jy[0].T[1:201])
n = np.arange(mean_list_pso.shape[0])

plt.plot(n, mean_list_pso, "--r", label="pso")
plt.plot(n, mean_list_jy, 'b', label="jaya")
plt.plot(n, mean_list_pso_jaya, ':g',  label="pso_jy")
plt.xlabel("iteration")
plt.ylabel("error")
plt.legend()
plt.grid('True')
plt.semilogy()


In [ ]:
df = pd.DataFrame({'PSO': all_err_pso[0].T[-1], 'Jaya': all_err_jy[0].T[-1], 'pso_jaya': all_err_pso_jy[0].T[1:201]})

#create boxplot by group
plt.boxplot(df, labels=['pso', 'jaya', 'pso-jaya'])
plt.xlabel("algorithm")
plt.ylabel("error")
plt.semilogy()